In [ ]:
# using property class
class Celsius:
    def __init__(self, temperature=0):
        self.temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    # getter
    @property
    def temperature(self):
        print("Getting value...")
        return self._temperature

    # setter

    @temperature.setter
    def temperature(self, value):
        print("Setting value...")
        if value < -273.15:
            raise ValueError("Temperature below -273.15 is not possible")
        self._temperature = value

    # creating a property object
    #temperature = property(get_temperature, set_temperature)


human = Celsius(-273)
human.temperature
#human.to_fahrenheit()

#human.temperature = -300

In [ ]:
import numpy as np

l = 100
eg = np.linspace(-7, 7, num=l)
#eg = np.random.shuffle(eg)

rng = np.random.default_rng()
rng.shuffle(eg)
#print(eg)
print(eg)

In [ ]:
import pandas as pd
from datetime import datetime

path = r'/Users/sr2/OneDrive - University College London/PhD/Research/Missions/SWARM/Non-Flight Data/Analysis/Apr-22/data/extraction/mag/MAG-data_April-16.csv'
df = pd.read_csv(path)

df = df.drop(columns=['nec_x','nec_y','nec_z'])

df=df.iloc[::20000]

def splitDatetime(df):
    #Splits datetime into date & utc, then reorders the df
    temp_df = df["datetime"].str.split(" ", n = 1, expand = True)
    df["date"] = temp_df [0]
    df["utc"] = temp_df [1]
    #df = df.reset_index().drop(columns=['datetime','index'])
    return df

df['alt'] = (df.pop('alt') / 1000 ) - 6371

def doy(x):
    day_of_year = x.timetuple().tm_yday
    return day_of_year
    
df = splitDatetime(df)
df['datetime'] = pd.to_datetime(df.pop('datetime'), format='%Y-%m-%d %H:%M:%S')
df['doy'] = df['datetime'].apply(doy)

def dns(x):
    dns = np.sin((2*np.pi*x)/365)
    return dns

def dnc(x):
    dnc = np.cos((2*np.pi*x)/365)
    return dnc

df['dns'] = df['doy'].apply(dns)
df['dnc'] = df['doy'].apply(dnc)

df.shape

In [ ]:
import requests
import xml.etree.ElementTree as ET
import datetime

def get_igrf(col):

    lat = col['lat']
    lon = col['long']
    alt = col['alt']
    date = col['date']
    
    path = 'https://geomag.bgs.ac.uk/web_service/GMModels/igrf/13/?latitude='+str(lat)+'&longitude='+str(lon)+'&altitude='+str(alt)+'&date='+str(date)+'&format=xml'
    
    response = requests.get(path)

    root = ET.fromstring(response.content)
    date = root[1].text

    for coors in root.findall('coordinates'):
        _lat = coors.find('latitude').text
        _lon = coors.find('longitude').text

    for fields in root.findall('field-value'):
        _ti = fields.find('total-intensity').text
        _dec = fields.find('declination').text
        _inc = fields.find('inclination').text
        _ni = fields.find('north-intensity').text
        _ei = fields.find('east-intensity').text
        _vi = fields.find('vertical-intensity').text
        _hi = fields.find('horizontal-intensity').text

    for secular in root.findall('secular-variation'):
        s_ti = secular.find('total-intensity').text
        s_dec = secular.find('declination').text
        s_inc = secular.find('inclination').text
        s_ni = secular.find('north-intensity').text
        s_ei = secular.find('east-intensity').text
        s_vi = secular.find('vertical-intensity').text
        s_hi = secular.find('horizontal-intensity').text

    #return pd.Series([_ti, _dec, _inc, _ni, _ei, _vi, _hi,
    #        s_ti, s_dec, s_inc, s_ni, s_ei, s_vi, s_hi])
    
    return pd.Series([_ti,_dec])
    #return _ti

df[['ti','dec']] = df.apply(get_igrf, axis=1)
print(df.head(5))

#print('Starting IGRF extraction at', pre_time)
#pre_time = datetime.datetime.now()
#df[['ti','dec','inc','ni','ei','vi','hi','s_ti','s_dec','s_inc','s_ni','s_ei','s_vi','s_hi']]= df.apply(get_igrf, axis=1)
#post_time = datetime.datetime.now()
#print('Finished IGRF extraction at', post_time)
#diff_time = post_time - pre_time
#print('Processing time:', diff_time)
#print(df.head())
#print(df.describe().transpose())



In [ ]:
from ftplib import FTP
import pandas as pd

ktp = FTP('ftp.gfz-potsdam.de')
ktp.login()
ktp.cwd('/pub/home/obs/Kp_ap_Ap_SN_F107')
years = [2014,2015,2016,2017,2018,2018,2019,2020,2021]
for i in years:
    filename = 'Kp_ap_'+str(i)+'.txt'
    with open(filename, 'wb' ) as file :
        ktp.retrbinary('RETR %s' % filename, file.write)

        file.close()
    print(f'Downloading data for {i}')

ktp.quit()

In [315]:
from pathlib import Path
import pandas as pd
import numpy as np
import datetime

path = Path(r'/Users/sr2/OneDrive - University College London/PhD/Research/'
        'Missions/SWARM/Non-Flight Data/Analysis/Mar-22/data/solar_max/')

dir_suffix = '2014'
#pre_classified = str(path) + '/' + dir_suffix +'-data-2022-03-03.csv'
post_classified = str(path) + '/ml_model/SG-filtered_14-15.csv'

def open_all(filename):
    print('Loading data...')
    df = pd.read_csv(filename)
    return df

df = open_all(post_classified)
#df = df.iloc[::10000]

Loading data...


In [316]:
#df['dt'] = df['date'] +' '+ df['utc']
#df = df[['dt','lat','long','date','utc']]

get_hr = df['utc'].str.slice(stop=2).astype(float)
#df['hr'] = get_hr

def round_down(num):
    return num - (num%3)

df['hr'] = get_hr.apply(round_down)
df.dtypes

date          object
utc           object
mlt          float64
lat          float64
long         float64
alt          float64
s_id          object
p_num          int64
b_ind          int64
b_prob       float64
Ne           float64
Ne_std       float64
Ti           float64
Ti_std       float64
pot          float64
pot_std      float64
Te           float64
Te_std       float64
Ne_savgol    float64
Ne_resid     float64
sg_epb         int64
sg_smooth      int64
Ne_pc        float64
Ne_stddev    float64
hr           float64
dtype: object

In [317]:
years = [2014,2015,2016,2017,2018,2018,2019,2020,2021]
listing = []
for i in years:
    filename = 'Kp_ap_'+str(i)+'.txt'
    kp_df = pd.read_csv(filename, on_bad_lines='skip', sep=" ")
    kp_df = kp_df.iloc[18:].reset_index().drop(columns=['level_0','index','ap','index.1','three-hour','interval)','distributes','the','Kp','per'])
    kp_df = kp_df.rename(columns={'#':'yy','PURPOSE:':'mm','This':'dd','file':'hr','and':'Kp','(one':'ap1','line':'ap2'})
    kp_df['date'] = kp_df['yy'] +'-'+ kp_df['mm'] +'-'+ kp_df['dd']
    kp_df = kp_df.drop(['yy', 'mm','dd'], axis=1)
    kp_df['hr']= kp_df['hr'].astype(float)

    kp_df['ap2']= kp_df['ap2'].mask(kp_df['ap2'] == 1, np.nan)
    kp_df['ap']=kp_df['ap2'].fillna(kp_df['ap1'])
    kp_df = kp_df.drop(columns=['ap1','ap2'])

    listing.append(kp_df)

kp_df = pd.concat(listing)
kp_df.dtypes


hr      float64
Kp       object
date     object
ap       object
dtype: object

In [311]:

filename="Kp_ap_2015.txt"
kp_df = pd.read_csv(filename, on_bad_lines='skip', sep=" ")

kp_df = kp_df.iloc[18:].reset_index().drop(columns=['level_0','index','ap','index.1','three-hour','interval)','distributes','the','Kp','per'])
kp_df = kp_df.rename(columns={'#':'yy','PURPOSE:':'mm','This':'dd','file':'hr','and':'Kp','(one':'ap1','line':'ap2'})
kp_df['date'] = kp_df['yy'] +'-'+ kp_df['mm'] +'-'+ kp_df['dd']
kp_df = kp_df.drop(['yy', 'mm','dd'], axis=1)
#kp_df=kp_df.apply(pd.to_numeric)
#kp_df = 

kp_df['hr']= kp_df['hr'].astype(float)

kp_df['ap2']= kp_df['ap2'].mask(kp_df['ap2'] == 1, np.nan)
kp_df['ap']=kp_df['ap2'].fillna(kp_df['ap1'])
kp_df = kp_df.drop(columns=['ap1','ap2'])
kp_df.dtypes




hr      float64
Kp       object
date     object
ap       object
dtype: object

In [318]:
df3 = pd.merge(df, kp_df, on=['hr','date'])
df3

,date,utc,mlt,lat,long,alt,s_id,p_num,b_ind,b_prob,...,Te_std,Ne_savgol,Ne_resid,sg_epb,sg_smooth,Ne_pc,Ne_stddev,hr,Kp,ap
0,2014-09-16,00:54:48,0.340303,33.678443,-15.371701,6837779.69,A,34,0,0.0,...,0.001571,284660.992298,342.807702,0,0,0.006247,0.001851,0.0,1.667,6
1,2014-09-16,00:54:49,0.339955,33.614549,-15.371627,6837791.43,A,34,0,0.0,...,0.001391,285922.815901,-106.915901,0,0,0.002849,0.001885,0.0,1.667,6
2,2014-09-16,00:54:50,0.339607,33.550654,-15.371560,6837803.18,A,34,0,0.0,...,0.001454,287184.048696,408.751304,0,0,0.006217,0.001788,0.0,1.667,6
3,2014-09-16,00:54:51,0.339259,33.486760,-15.371499,6837814.93,A,34,0,0.0,...,0.001394,288401.363975,336.236025,0,0,0.003981,0.001625,0.0,1.667,6
4,2014-09-16,00:54:52,0.338911,33.422866,-15.371445,6837826.68,A,34,0,0.0,...,0.001385,289622.914907,95.485093,0,0,0.003397,0.001666,0.0,1.667,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1883676,2015-02-02,23:59:55,22.923583,-4.906024,-8.412674,6838589.92,A,3528,0,0.0,...,0.004571,574375.967504,-16.167504,0,0,0.005506,0.002330,21.0,3.000,1
1883677,2015-02-02,23:59:56,22.924578,-4.842117,-8.413883,6838579.48,A,3528,0,0.0,...,0.004538,577885.924167,-455.324167,0,0,0.005346,0.002330,21.0,3.000,1
1883678,2015-02-02,23:59:57,22.925570,-4.778210,-8.415092,6838569.04,A,3528,0,0.0,...,0.004500,581493.796369,1049.903631,0,0,0.008855,0.002472,21.0,3.000,1
1883679,2015-02-02,23:59:58,22.926559,-4.714303,-8.416301,6838558.58,A,3528,0,0.0,...,0.004444,585199.584111,384.215889,0,0,0.005219,0.002472,21.0,3.000,1
